In [1]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# pylint: skip-file
# pyformat: disable

import functools
import os
import shutil
from typing import Callable
import json

import jax
import jax.numpy as jn

import numpy as np
import tensorflow as tf  # For data augmentation.
import tensorflow_datasets as tfds
from absl import app, flags

import objax
from objax.jaxboard import SummaryWriter, Summary
from objax.util import EasyDict
from objax.zoo import convnet, wide_resnet

from dataset import DataSet


############################# from MODEL ZOO
import torch

import torchvision.transforms as transforms

from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler

from torch.utils.data import Subset


FLAGS = flags.FLAGS

2024-10-10 21:32:04.081210: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-10 21:32:04.168407: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-10 21:32:04.193869: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-10 21:32:06.105441: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [49]:
def augment(x, shift: int, mirror=True):
    """
    Augmentation function used in training the model.
    """
    y = x['image']
    if mirror:
        y = tf.image.random_flip_left_right(y)
    y = tf.pad(y, [[shift] * 2, [shift] * 2, [0] * 2], mode='REFLECT')
    y = tf.image.random_crop(y, tf.shape(x['image']))
    return dict(image=y, label=x['label'])


class TrainLoop(objax.Module):
    """
    Training loop for general machine learning models.
    Based on the training loop from the objax CIFAR10 example code.
    """
    predict: Callable
    train_op: Callable

#     def __init__(self, nclass: int, **kwargs):
#         self.nclass = nclass
#         self.params = EasyDict(kwargs)

#     def train_step(self, summary: Summary, data: dict, progress: np.ndarray):
#         kv = self.train_op(progress, data['image'].numpy(), data['label'].numpy())
#         for k, v in kv.items():
#             if jn.isnan(v):
#                 raise ValueError('NaN, try reducing learning rate', k)
#             if isinstance(v, jn.ndarray) and v.ndim > 0:
#                 v = v.item()  # Converts array to a scalar if it's a single value array
                
#             if summary is not None:
#                 summary.scalar(k, float(v))

    def train(self, num_train_epochs: int, train_size: int, train: DataSet, test: DataSet, logdir: str, save_steps=100, patience=None):
        """
        Completely standard training. Nothing interesting to see here.
        """
        checkpoint = objax.io.Checkpoint(logdir, keep_ckpts=20, makedir=True)
        start_epoch, last_ckpt = checkpoint.restore(self.vars())
        train_iter = iter(train)
        progress = np.zeros(jax.local_device_count(), 'f')  # for multi-GPU

        best_acc = 0
        best_acc_epoch = -1

        with SummaryWriter(os.path.join(logdir, 'tb')) as tensorboard:
            for epoch in range(start_epoch, num_train_epochs):
                # Train
                summary = Summary()
                loop = range(0, train_size, self.params.batch)
                for step in loop:
                    progress[:] = (step + (epoch * train_size)) / (num_train_epochs * train_size)
                    
                    next_iter = next(train_iter)
                    print("next_iter: ", next_iter.shape)
                    
                    self.train_step(summary, next_iter, progress)

                Eval
                accuracy, total = 0, 0
                if epoch%FLAGS.eval_steps == 0 and test is not None:
                    for data in test:
                        total += data['image'].shape[0]
                        preds = np.argmax(self.predict(data['image'].numpy()), axis=1)
                        accuracy += (preds == data['label'].numpy()).sum()
                    accuracy /= total
                    summary.scalar('eval/accuracy', 100 * accuracy)
                    tensorboard.write(summary, step=(epoch + 1) * train_size)
                    print('Epoch %04d  Loss %.2f  Accuracy %.2f' % (epoch + 1, summary['losses/xe'](),
                                                                    summary['eval/accuracy']()))

                    if summary['eval/accuracy']() > best_acc:
                        best_acc = summary['eval/accuracy']()
                        best_acc_epoch = epoch
                    elif patience is not None and epoch > best_acc_epoch + patience:
                        print("early stopping!")
                        checkpoint.save(self.vars(), epoch + 1)
                        return

                else:
                    print('Epoch %04d  Loss %.2f  Accuracy --' % (epoch + 1, summary['losses/xe']()))

                if epoch%save_steps == save_steps-1:
                    checkpoint.save(self.vars(), epoch + 1)


# We inherit from the training loop and define predict and train_op.
class MemModule(TrainLoop):
    def __init__(self, model: Callable, nclass: int, mnist=False, **kwargs):
        """
        Completely standard training. Nothing interesting to see here.
        """
        super().__init__(nclass, **kwargs)
        self.model = model(1 if mnist else 3, nclass)
        self.opt = objax.optimizer.Momentum(self.model.vars())
        self.model_ema = objax.optimizer.ExponentialMovingAverageModule(self.model, momentum=0.999, debias=True)

        @objax.Function.with_vars(self.model.vars())
#         def loss(x, label):
#             logit = self.model(x, training=True)
#             loss_wd = 0.5 * sum((v.value ** 2).sum() for k, v in self.model.vars().items() if k.endswith('.w'))
#             loss_xe = objax.functional.loss.cross_entropy_logits(logit, label).mean()
#             return loss_xe + loss_wd * self.params.weight_decay, {'losses/xe': loss_xe, 'losses/wd': loss_wd}

        gv = objax.GradValues(loss, self.model.vars())
        self.gv = gv

        @objax.Function.with_vars(self.vars())
#         def train_op(progress, x, y):
#             g, v = gv(x, y)
#             lr = self.params.lr * jn.cos(progress * (7 * jn.pi) / (2 * 8))
#             lr = lr * jn.clip(progress*100,0,1)
#             self.opt(lr, g)
#             self.model_ema.update_ema()
#             return {'monitors/lr': lr, **v[1]}

        self.predict = objax.Jit(objax.nn.Sequential([objax.ForceArgs(self.model_ema, training=False)]))

        self.train_op = objax.Jit(train_op)


# def network(arch: str):
#     if arch == 'cnn32-3-max':
#         return functools.partial(convnet.ConvNet, scales=3, filters=32, filters_max=1024,
#                                  pooling=objax.functional.max_pool_2d)
#     elif arch == 'cnn32-3-mean':
#         return functools.partial(convnet.ConvNet, scales=3, filters=32, filters_max=1024,
#                                  pooling=objax.functional.average_pool_2d)
#     elif arch == 'cnn64-3-max':
#         return functools.partial(convnet.ConvNet, scales=3, filters=64, filters_max=1024,
#                                  pooling=objax.functional.max_pool_2d)
#     elif arch == 'cnn64-3-mean':
#         return functools.partial(convnet.ConvNet, scales=3, filters=64, filters_max=1024,
#                                  pooling=objax.functional.average_pool_2d)
#     elif arch == 'wrn28-1':
#         return functools.partial(wide_resnet.WideResNet, depth=28, width=1)
#     elif arch == 'wrn28-2':
#         return functools.partial(wide_resnet.WideResNet, depth=28, width=2)
#     elif arch == 'wrn28-10':
#         return functools.partial(wide_resnet.WideResNet, depth=28, width=10)
#     raise ValueError('Architecture not recognized', arch)
    
def get_data(seed):
    """
    This is the function to generate subsets of the data for training models.

    First, we get the training dataset either from the numpy cache
    or otherwise we load it from tensorflow datasets.

    Then, we compute the subset. This works in one of two ways.

    1. If we have a seed, then we just randomly choose examples based on
       a prng with that seed, keeping FLAGS.pkeep fraction of the data.

    2. Otherwise, if we have an experiment ID, then we do something fancier.
       If we run each experiment independently then even after a lot of trials
       there will still probably be some examples that were always included
       or always excluded. So instead, with experiment IDs, we guarantee that
       after FLAGS.num_experiments are done, each example is seen exactly half
       of the time in train, and half of the time not in train.

    """
#     DATA_DIR = os.path.join(os.environ['HOME'], 'TFDS')
    
    if os.path.exists(os.path.join(FLAGS.logdir, "x_train.npy")):
        inputs = np.load(os.path.join(FLAGS.logdir, "x_train.npy"))
        labels = np.load(os.path.join(FLAGS.logdir, "y_train.npy"))
    else:
        print("First time, creating dataset")

        ### 
        transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        # https://huggingface.co/datasets/ILSVRC/imagenet-1k
        data_dir = '/serenity/scratch/psml/repo/psml/data/ILSVRC2012'
        imagenet_data = datasets.ImageNet(root=data_dir, split='val', transform=transform)
        
        data_loader = torch.utils.data.DataLoader(imagenet_data, batch_size=256, shuffle=False, num_workers=4)

        inputs_ = [] 
        labels_ = []
        iter = 0
        for images__, labels__ in data_loader:     
            print(iter)
            iter += 1
            inputs_.append(images__)
            labels_.append(labels__)

        inputs = np.concatenate(inputs_, axis=0)
        labels = np.concatenate(labels_, axis=0)
        ### 
        
        np.save(os.path.join(FLAGS.logdir, "x_train.npy"),inputs)
        np.save(os.path.join(FLAGS.logdir, "y_train.npy"),labels)

    nclass = np.max(labels)+1

    # TODO 
    # 1. split inputs and label to train / test datasets with the ratio 80:20.
    # 2. 
    
    np.random.seed(seed)
    if FLAGS.num_experiments is not None:
        np.random.seed(0)
        keep = np.random.uniform(0,1,size=(FLAGS.num_experiments, FLAGS.dataset_size))
        order = keep.argsort(0)
        keep = order < int(FLAGS.pkeep * FLAGS.num_experiments)
        keep = np.array(keep[FLAGS.expid], dtype=bool)
    else:
        keep = np.random.uniform(0, 1, size=FLAGS.dataset_size) <= FLAGS.pkeep

    if FLAGS.only_subset is not None:
        keep[FLAGS.only_subset:] = 0
        
    

    xs = inputs[keep]
    ys = labels[keep]

    if FLAGS.augment == 'weak':
        aug = lambda x: augment(x, 4)
    elif FLAGS.augment == 'mirror':
        aug = lambda x: augment(x, 0)
    elif FLAGS.augment == 'none':
        aug = lambda x: augment(x, 0, mirror=False)
    else:
        raise

    train = DataSet.from_arrays(xs, ys,
                                augment_fn=aug)
    test = DataSet.from_tfds(tfds.load(name=FLAGS.dataset, split='test', data_dir=DATA_DIR), xs.shape[1:])
    train = train.cache().shuffle(8192).repeat().parse().augment().batch(FLAGS.batch)
    train = train.nchw().one_hot(nclass).prefetch(16)
    test = test.cache().parse().batch(FLAGS.batch).nchw().prefetch(16)

    return train, test, xs, ys, keep, nclass

def main(argv):
    del argv
    tf.config.experimental.set_visible_devices([], "GPU")

    seed = FLAGS.seed
    if seed is None:
        import time
        seed = np.random.randint(0, 1000000000)
        seed ^= int(time.time())

    args = EasyDict(arch=FLAGS.arch,
                    lr=FLAGS.lr,
                    batch=FLAGS.batch,
                    weight_decay=FLAGS.weight_decay,
                    augment=FLAGS.augment,
                    seed=seed)


    if FLAGS.tunename:
        logdir = '_'.join(sorted('%s=%s' % k for k in args.items()))
    elif FLAGS.expid is not None:
        logdir = "experiment-%d_%d"%(FLAGS.expid,FLAGS.num_experiments)
    else:
        logdir = "experiment-"+str(seed)
    logdir = os.path.join(FLAGS.logdir, logdir)

    if os.path.exists(os.path.join(logdir, "ckpt", "%010d.npz"%FLAGS.epochs)):
        print(f"run {FLAGS.expid} already completed.")
        return
    else:
        if os.path.exists(logdir):
            print(f"deleting run {FLAGS.expid} that did not complete.")
            shutil.rmtree(logdir)

    print(f"starting run {FLAGS.expid}.")
    if not os.path.exists(logdir):
        os.makedirs(logdir)

    train, test, xs, ys, keep, nclass = get_data(seed)

    # Define the network and train_it
    tm = MemModule(network(FLAGS.arch), nclass=nclass,
                   mnist=FLAGS.dataset == 'mnist',
                   epochs=FLAGS.epochs,
                   expid=FLAGS.expid,
                   num_experiments=FLAGS.num_experiments,
                   pkeep=FLAGS.pkeep,
                   save_steps=FLAGS.save_steps,
                   only_subset=FLAGS.only_subset,
                   **args
    )

    r = {}
    r.update(tm.params)

    open(os.path.join(logdir,'hparams.json'),"w").write(json.dumps(tm.params))
    np.save(os.path.join(logdir,'keep.npy'), keep)

    tm.train(FLAGS.epochs, len(xs), train, test, logdir,
             save_steps=FLAGS.save_steps, patience=FLAGS.patience)

In [3]:
'''
if __name__ == '__main__':
    flags.DEFINE_string('arch', 'cnn32-3-mean', 'Model architecture.')
    flags.DEFINE_float('lr', 0.1, 'Learning rate.')
    flags.DEFINE_string('dataset', 'cifar10', 'Dataset.')
    flags.DEFINE_float('weight_decay', 0.0005, 'Weight decay ratio.')
    flags.DEFINE_integer('batch', 256, 'Batch size')
    flags.DEFINE_integer('epochs', 501, 'Training duration in number of epochs.')
    flags.DEFINE_string('logdir', 'experiments', 'Directory where to save checkpoints and tensorboard data.')
    flags.DEFINE_integer('seed', None, 'Training seed.')
    flags.DEFINE_float('pkeep', .5, 'Probability to keep examples.')
    flags.DEFINE_integer('expid', None, 'Experiment ID')
    flags.DEFINE_integer('num_experiments', None, 'Number of experiments')
    flags.DEFINE_string('augment', 'weak', 'Strong or weak augmentation')
    flags.DEFINE_integer('only_subset', None, 'Only train on a subset of images.')
    flags.DEFINE_integer('dataset_size', 50000, 'number of examples to keep.')
    flags.DEFINE_integer('eval_steps', 1, 'how often to get eval accuracy.')
    flags.DEFINE_integer('abort_after_epoch', None, 'stop trainin early at an epoch')
    flags.DEFINE_integer('save_steps', 10, 'how often to get save model.')
    flags.DEFINE_integer('patience', None, 'Early stopping after this many epochs without progress')
    flags.DEFINE_bool('tunename', False, 'Use tune name?')
    
    ### override 
    # https://github.com/tensorflow/privacy/tree/d965556ebb67bd62626830339478e9ebab7ab9bd/research/mi_lira_2021/scripts  
#     CUDA_VISIBLE_DEVICES='0' python3 -u train.py --dataset=cifar10 --epochs=100 --save_steps=20 --arch wrn28-2 --num_experiments 16 --expid 0 --logdir exp/cifar10 &> logs/log_0

    flags.DEFINE_string('dataset', 'cifar10', 'Dataset.')
    flags.DEFINE_string('arch', 'wrn28-2', 'Model architecture.')

    flags.DEFINE_integer('epochs', 20, 'Training duration in number of epochs.')
    flags.DEFINE_integer('save_steps', 5, 'how often to get save model.')
    flags.DEFINE_integer('num_experiments', 16, 'Number of experiments')
    flags.DEFINE_integer('expid', 3, 'Experiment ID')
    flags.DEFINE_string('logdir', 'experiments', 'Directory where to save checkpoints and tensorboard data.')
        
    app.run(main)

'''

"\nif __name__ == '__main__':\n    flags.DEFINE_string('arch', 'cnn32-3-mean', 'Model architecture.')\n    flags.DEFINE_float('lr', 0.1, 'Learning rate.')\n    flags.DEFINE_string('dataset', 'cifar10', 'Dataset.')\n    flags.DEFINE_float('weight_decay', 0.0005, 'Weight decay ratio.')\n    flags.DEFINE_integer('batch', 256, 'Batch size')\n    flags.DEFINE_integer('epochs', 501, 'Training duration in number of epochs.')\n    flags.DEFINE_string('logdir', 'experiments', 'Directory where to save checkpoints and tensorboard data.')\n    flags.DEFINE_integer('seed', None, 'Training seed.')\n    flags.DEFINE_float('pkeep', .5, 'Probability to keep examples.')\n    flags.DEFINE_integer('expid', None, 'Experiment ID')\n    flags.DEFINE_integer('num_experiments', None, 'Number of experiments')\n    flags.DEFINE_string('augment', 'weak', 'Strong or weak augmentation')\n    flags.DEFINE_integer('only_subset', None, 'Only train on a subset of images.')\n    flags.DEFINE_integer('dataset_size', 5000

# import imagenet using tfds

In [4]:
for ds in tfds.list_builders(): 
    if 'imagenet' in ds:
        print(ds)

downsampled_imagenet
imagenet2012
imagenet2012_corrupted
imagenet2012_fewshot
imagenet2012_multilabel
imagenet2012_real
imagenet2012_subset
imagenet_a
imagenet_lt
imagenet_pi
imagenet_r
imagenet_resized
imagenet_sketch
imagenet_v2
imagenette
huggingface:imagenet-1k
huggingface:imagenet_sketch


In [29]:
DATA_DIR = os.path.join(os.environ['HOME'], 'TFDS')

data = tfds.as_numpy(tfds.load(name='cifar10', batch_size=-1, data_dir=DATA_DIR))
inputs = data['train']['image']
labels = data['train']['label']

inputs = (inputs/127.5)-1

nclass = np.max(labels)+1

seed = 128 # customization 
num_experiments = 16
dataset_size = 50000
pkeep = 0.5
expid = 1
only_subset = None
batch = 32

np.random.seed(seed)
if num_experiments is not None:
    np.random.seed(0)
    keep = np.random.uniform(0,1,size=(num_experiments, dataset_size))
    order = keep.argsort(0)
    keep = order < int(pkeep * num_experiments)
    keep = np.array(keep[expid], dtype=bool)
else:
    keep = np.random.uniform(0, 1, size=dataset_size) <= pkeep

if only_subset is not None:
    keep[only_subset:] = 0

xs = inputs[keep]
ys = labels[keep]

# if FLAGS.augment == 'weak':
#     aug = lambda x: augment(x, 4)
# elif FLAGS.augment == 'mirror':
#     aug = lambda x: augment(x, 0)
# elif FLAGS.augment == 'none':
#     aug = lambda x: augment(x, 0, mirror=False)
# else:
#     raise

aug = lambda x: augment(x, 0, mirror=False)

train = DataSet.from_arrays(xs, ys,
                            augment_fn=aug)
test = DataSet.from_tfds(tfds.load(name='cifar10', split='test', data_dir=DATA_DIR), xs.shape[1:])
train = train.cache().shuffle(8192).repeat().parse().augment().batch(batch)
train = train.nchw().one_hot(nclass).prefetch(16)
test = test.cache().parse().batch(batch).nchw().prefetch(16)

In [30]:
xs.shape

(25046, 32, 32, 3)

In [33]:
xs[0]

array([[[ 0.59215686,  0.67843137,  0.83529412],
        [ 0.49803922,  0.62352941,  0.77254902],
        [ 0.39607843,  0.56862745,  0.75686275],
        ...,
        [-0.00392157,  0.34901961,  0.67058824],
        [-0.01176471,  0.34117647,  0.6627451 ],
        [-0.02745098,  0.33333333,  0.65490196]],

       [[ 0.60784314,  0.67843137,  0.80392157],
        [ 0.45882353,  0.56078431,  0.67058824],
        [ 0.41176471,  0.54509804,  0.67843137],
        ...,
        [ 0.03529412,  0.39607843,  0.71764706],
        [ 0.01960784,  0.38039216,  0.71764706],
        [ 0.01176471,  0.37254902,  0.70196078]],

       [[ 0.51372549,  0.56862745,  0.67058824],
        [ 0.10588235,  0.18431373,  0.24705882],
        [-0.02745098,  0.0745098 ,  0.1372549 ],
        ...,
        [ 0.06666667,  0.39607843,  0.70980392],
        [ 0.05098039,  0.38823529,  0.70980392],
        [ 0.03529412,  0.38039216,  0.70196078]],

       ...,

       [[-0.68627451, -0.63137255, -0.56078431],
        [-0

In [32]:
ys.shape

(25046,)

In [3]:
for k,v in data['train'].items():
    print(k)

id
image
label


In [24]:
type(data['train']['id'])

numpy.ndarray

In [20]:
data['train']['id'].shape

(50000,)

In [18]:
data['train']['id']

array([b'train_16399', b'train_01680', b'train_47917', ...,
       b'train_05073', b'train_39601', b'train_32842'], dtype=object)

In [21]:
data['train']['image'].shape

(50000, 32, 32, 3)

In [17]:
data['train']['image']

array([[[[143,  96,  70],
         [141,  96,  72],
         [135,  93,  72],
         ...,
         [ 96,  37,  19],
         [105,  42,  18],
         [104,  38,  20]],

        [[128,  98,  92],
         [146, 118, 112],
         [170, 145, 138],
         ...,
         [108,  45,  26],
         [112,  44,  24],
         [112,  41,  22]],

        [[ 93,  69,  75],
         [118,  96, 101],
         [179, 160, 162],
         ...,
         [128,  68,  47],
         [125,  61,  42],
         [122,  59,  39]],

        ...,

        [[187, 150, 123],
         [184, 148, 123],
         [179, 142, 121],
         ...,
         [198, 163, 132],
         [201, 166, 135],
         [207, 174, 143]],

        [[187, 150, 117],
         [181, 143, 115],
         [175, 136, 113],
         ...,
         [201, 164, 132],
         [205, 168, 135],
         [207, 171, 139]],

        [[195, 161, 126],
         [187, 153, 123],
         [186, 151, 128],
         ...,
         [212, 177, 147],
        

In [54]:
data['train']['label']

array([7, 8, 4, ..., 8, 0, 4])

# from model zoo

In [2]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [3]:
# https://huggingface.co/datasets/ILSVRC/imagenet-1k

data_dir = '/serenity/scratch/psml/repo/psml/data/ILSVRC2012'

imagenet_data = datasets.ImageNet(root=data_dir, split='val', transform=transform)
data_loader = torch.utils.data.DataLoader(imagenet_data, batch_size=256, shuffle=False, num_workers=4)

In [4]:
imagenet_data

Dataset ImageNet
    Number of datapoints: 50000
    Root location: /serenity/scratch/psml/repo/psml/data/ILSVRC2012
    Split: val
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [34]:
imagenet_inputs = [] 
imagenet_labels = []

iter = 0
for images__, labels__ in data_loader:     
    print(iter)
    iter += 1
    imagenet_inputs.append(images__)
    imagenet_labels.append(labels__)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195


In [35]:
imagenet_inputs = np.concatenate(inputs_, axis=0)

In [36]:
imagenet_labels = np.concatenate(labels_, axis=0)

In [48]:
imagenet_inputs.shape

(50000, 3, 224, 224)

In [ ]:
for images, labels in data_loader:
    print(images)
    print(labels)

In [22]:
np.max(labels)+1

1000

In [39]:
model = functools.partial(convnet.ConvNet, scales=3, filters=32, filters_max=1024,
                                 pooling=objax.functional.max_pool_2d)

In [42]:
model.parameters()

AttributeError: 'functools.partial' object has no attribute 'parameters'

In [44]:
m = model(3, 1000)

In [45]:
m

objax.zoo.convnet.ConvNet(
  [0] objax.nn.Conv2D(nin=3, nout=32, k=(3, 3), strides=(1, 1), dilations=(1, 1), groups=1, padding='SAME', use_bias=True, w_init=kaiming_normal(*, gain=1))
  [1] leaky_relu(*, negative_slope=0.01)
  [2] objax.nn.Conv2D(nin=32, nout=32, k=(3, 3), strides=(1, 1), dilations=(1, 1), groups=1, padding='SAME', use_bias=True, w_init=kaiming_normal(*, gain=1))
  [3] leaky_relu(*, negative_slope=0.01)
  [4] objax.nn.Conv2D(nin=32, nout=64, k=(3, 3), strides=(1, 1), dilations=(1, 1), groups=1, padding='SAME', use_bias=True, w_init=kaiming_normal(*, gain=1))
  [5] leaky_relu(*, negative_slope=0.01)
  [6] max_pool_2d(*, size=2, strides=2, padding=ConvPadding.VALID)
  [7] objax.nn.Conv2D(nin=64, nout=64, k=(3, 3), strides=(1, 1), dilations=(1, 1), groups=1, padding='SAME', use_bias=True, w_init=kaiming_normal(*, gain=1))
  [8] leaky_relu(*, negative_slope=0.01)
  [9] objax.nn.Conv2D(nin=64, nout=128, k=(3, 3), strides=(1, 1), dilations=(1, 1), groups=1, padding='SAME', u

In [2]:
!CUDA_VISIBLE_DEVICES='0' python3 -u train.py --dataset=cifar10 --epochs=2 --save_steps=2 --arch wrn28-2 --num_experiments 16 --expid 9 --logdir exp/cifar10

2024-10-10 22:48:26.829435: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-10 22:48:26.842383: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-10 22:48:26.846202: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-10 22:48:28.257276: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
starting run 9.
I1010 22:48:31.987497 140313265325888 dataset_info.py:617] Load dataset info from /nethome/dsanyal7/TFDS/cifar10/3.0.2
I1010 22:48:31.991467 140313265325888 dataset_builder.py:579] Reusing dataset cifar10 (/nethome/dsanyal7/T

data['image'].numpy() : (256, 3, 32, 32)
data['label'].numpy() : (256, 10)
data['image'].numpy() : (256, 3, 32, 32)
data['label'].numpy() : (256, 10)
data['image'].numpy() : (256, 3, 32, 32)
data['label'].numpy() : (256, 10)
data['image'].numpy() : (256, 3, 32, 32)
data['label'].numpy() : (256, 10)
data['image'].numpy() : (256, 3, 32, 32)
data['label'].numpy() : (256, 10)
data['image'].numpy() : (256, 3, 32, 32)
data['label'].numpy() : (256, 10)
data['image'].numpy() : (256, 3, 32, 32)
data['label'].numpy() : (256, 10)
data['image'].numpy() : (256, 3, 32, 32)
data['label'].numpy() : (256, 10)
data['image'].numpy() : (256, 3, 32, 32)
data['label'].numpy() : (256, 10)
data['image'].numpy() : (256, 3, 32, 32)
data['label'].numpy() : (256, 10)
data['image'].numpy() : (256, 3, 32, 32)
data['label'].numpy() : (256, 10)
data['image'].numpy() : (256, 3, 32, 32)
data['label'].numpy() : (256, 10)
data['image'].numpy() : (256, 3, 32, 32)
data['label'].numpy() : (256, 10)
data['image'].numpy() : (

data['image'].numpy() : (256, 3, 32, 32)
data['label'].numpy() : (256, 10)
data['image'].numpy() : (256, 3, 32, 32)
data['label'].numpy() : (256, 10)
data['image'].numpy() : (256, 3, 32, 32)
data['label'].numpy() : (256, 10)
data['image'].numpy() : (256, 3, 32, 32)
data['label'].numpy() : (256, 10)
data['image'].numpy() : (256, 3, 32, 32)
data['label'].numpy() : (256, 10)
data['image'].numpy() : (256, 3, 32, 32)
data['label'].numpy() : (256, 10)
data['image'].numpy() : (256, 3, 32, 32)
data['label'].numpy() : (256, 10)
data['image'].numpy() : (256, 3, 32, 32)
data['label'].numpy() : (256, 10)
data['image'].numpy() : (256, 3, 32, 32)
data['label'].numpy() : (256, 10)
Epoch 0002  Loss 1.41  Accuracy 43.81
